<a href="https://colab.research.google.com/github/adalves-ufabc/2021.QS-PLN/blob/main/2021_Q1_PLN_Notebook_36.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2021.Q1]**
Prof. Alexandre Donizeti Alves

### **Reconhecimento de Entidades Nomeadas** 
----

Existem diversas aplicações que podem fazer uso dessa técnica:

* **Resumo de grandes documentos**: um sistema que usa NER poderia aprender a resumir textos grandes em pequenos a partir do conhecimento de entidades relevantes.
* **Categorização de *reviews***: ao aplicar NER em comentários de usuários em portais, pode se gerar categorias que resumem todo o texto.
* **Chatbot de auto-atendimento**: diversos chatbots estão sendo construídos nos quais a base para seu funcionamento é a técnica de NER. Com o reconhecimento das entidades os bots tomam ações propícias conforme o usuário fala ou digita.
* **Identificação e agendamento de compromissos**: usando NER um sistema consegue aprender o que é data, horário e uma cidade em um e-mail por exemplo. Com essas informações, o sistema poderia agendar um compromisso automaticamente na sua agenda.




Primeiramente precisamos fazer download de alguns modelos prontos. Esses modelos contêm informações sobre linguagens, vocabulários, vetores treinados, sintaxes e entidades.

>
Vamos inicar fazendo o download dos modelos para o idioma Português e Inglês:

In [1]:
!python -m spacy download pt
!python -m spacy download en

     |████████████████████████████████| 21.2MB 83.2MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp37-none-any.whl size=21186283 sha256=8cb856a19f137ab454fa812eaad2d653e3eb1e48ced7088f08af68e9f81fca4a
  Stored in directory: /tmp/pip-ephem-wheel-cache-qbb8h2yv/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


Em seguida, carregamos o modelo português e passamos uma sentença para o método **`nlp()`** para este iniciar o processamento do texto.

In [2]:
import spacy

nlp = spacy.load('pt')
texto = nlp('Olá Mundo. Clássica frase no mundo da computação')
texto

Olá Mundo. Clássica frase no mundo da computação

In [3]:
type(texto)

spacy.tokens.doc.Doc

Cada palavra da sentença para o spaCy é um token. Isso significa que esse objeto contém propriedades interessantes. Vejamos alguns exemplos:

In [4]:
for token in texto:
  print(token, token.idx, token.shape_, token.tag_)

Olá 0 Xxx PROPN
Mundo 4 Xxxxx PROPN
. 9 . PU|@PU
Clássica 11 Xxxxx PROP|F|S|@SUBJ>
frase 20 xxxx <np-def>|N|F|S|@SUBJ>
no 26 xx PRP|@N<
mundo 29 xxxx <np-idf>|N|M|S|@P<
da 35 xx PRP|@N<
computação 38 xxxx <np-idf>|N|F|S|@P<


A propriedade **`.idx`** retorna o índice de inicio do token (palavra).

A propriedade **`.shape`**_ mostra o formato da palavra, se esta começa com letra maiúscula tem uma forma diferente, por exemplo.

Já a propriedade **`.tag_`** mostra informações gramáticais como ‘PROPN’ que significa ‘Proper Noun’, ou seja, substantivo próprio.

In [5]:
texto = nlp('O rato roeu a roupa do rei de Roma')

for entidade in texto.ents:
  print(entidade.text, entidade.label_)

Roma LOC


In [6]:
texto = nlp('Maria está se mudando para Paris. No dia 25/12/2020 ela irá partir')

Temos agora 3 entidades que queremos identificar, no caso a pessoa "Maria", a cidade de "Paris" e a data "25/12/2020".

In [7]:
from spacy import displacy
displacy.render(texto, style='ent', jupyter=True)

In [8]:
spacy.explain('PER')

'Named person or family.'

Notem que o modelo não identificou a `data` na sentença.

In [8]:
texto = nlp('Data de Nascimento: 21/03/1960')

displacy.render(texto, style='ent', jupyter=True);

In [9]:
texto = nlp('No dia 03/11/2020 a Joana foi aprovada no concurso')

displacy.render(texto, style='ent', jupyter=True);

Nos dois exemplos acima, o modelo também não identificou as entidades `data` nas sentenças.

### **Como treinar NER a partir de um modelo spaCy vazio**

Vamos analisar o seguinte exemplo:

In [10]:
import spacy
from spacy import displacy

nlp = spacy.load('en_core_web_sm')

doc = nlp("I ate Sushi yesterday. Maggi is a common fast food")

displacy.render(doc, style='ent', jupyter=True)

Se você não quiser usar um modelo pré-existente, você pode criar um modelo vazio usando `spacy.blank()` apenas passando o ID do idioma. Para criar um modelo vazio no idioma inglês, você deve passar "en".

O código a seguir mostra as etapas iniciais para treinar o NER de um novo modelo vazio:

In [11]:
import spacy

nlp=spacy.blank("en")

nlp.add_pipe(nlp.create_pipe('ner'))

nlp.begin_training()

Depois disso, você pode seguir exatamente o mesmo procedimento como no caso do modelo pré-existente. É assim que você pode treinar o reconhecedor de entidade nomeada para identificar e categorizar corretamente de acordo com o contexto.

E se você quiser colocar uma entidade em uma categoria que ainda não está presente?

Considere que você tem muitos dados de texto sobre os alimentos consumidos em diversas áreas. E você deseja que o NER classifique todos os itens alimentares na categoria **`FOOD`**. Mas, não existe tal categoria.

`spaCy` é altamente flexível e permite adicionar um novo tipo de entidade e treinar o modelo. Esse recurso é extremamente útil, pois permite adicionar novos tipos de entidades para facilitar a recuperação de informações.

Primeiro, carregue o modelo de espaço pré-existente que deseja usar e obtenha o pipeline `ner` por meio do método `get_pipe()`.

In [12]:
# import and load the spacy model
import spacy
nlp=spacy.load("en_core_web_sm") 

# Getting the ner component
ner=nlp.get_pipe('ner')

Em seguida, armazene o nome do novo tipo de categoria/entidade em uma variável de string LABEL.

Agora, como o modelo saberá quais entidades devem ser classificadas sob o novo rótulo?

Você terá que treinar o modelo com exemplos. Os exemplos de treinamento devem ensinar ao modelo que tipo de entidades devem ser classificadas como `FOOD`.

O formato dos dados de treinamento é uma lista de tuplas. Cada tupla contém o texto de exemplo e um dicionário. O dicionário terá as entidades chave, que armazenam os índices inicial e final junto com o rótulo das entidades presentes no texto.

>
Por exemplo, para passar "Pizza is a common fast food" como exemplo, o formato será: 
 > `("Pizza is a common fast food", {"entities": [(0, 5, "FOOD")]})`.

In [13]:
# new label to add
LABEL = "FOOD"

# Training examples in the required format
TRAIN_DATA =[ ("Pizza is a common fast food.", {"entities": [(0, 5, LABEL)]}),
              ("Pasta is an italian recipe", {"entities": [(0, 5, LABEL)]}),
              ("China's noodles are very famous", {"entities": [(8,14, LABEL)]}),
              ("Shrimps are famous in China too", {"entities": [(0,7, LABEL)]}),
              ("Lasagna is another classic of Italy", {"entities": [(0,7, LABEL)]}),
              ("Sushi is extemely famous and expensive Japanese dish", {"entities": [(0,5, LABEL)]}),
              ("Unagi is a famous seafood of Japan", {"entities": [(0,5, LABEL)]}),
              ("Tempura , Soba are other famous dishes of Japan", {"entities": [(0,7, LABEL)]}),
              ("Chocolate soufflé is extremely famous french cuisine", {"entities": [(0,17, LABEL)]}),
              ("Flamiche is french pastry", {"entities": [(0,8, LABEL)]}),
              ("Burgers are the most commonly consumed fastfood", {"entities": [(0,7, LABEL)]}),
              ("Burgers are the most commonly consumed fastfood", {"entities": [(0,7, LABEL)]}),
              ("Frenchfries are considered too oily", {"entities": [(0,11, LABEL)]})
           ]

Agora que os dados de treinamento estão prontos, podemos prosseguir para ver como esses exemplos são usados para treinar o NER.

Lembre-se de que o rótulo “FOOD” não é conhecido pelo modelo agora.

Portanto, nossa primeira tarefa será adicionar o rótulo ao NER através do método `add_label()`. Em seguida, você pode usar a função `resume_training()` para retornar um otimizador.

Além disso, quando o treinamento é concluído, os outros componentes do pipeline também são afetados. Para evitar isso, use o método `disable_pipes()` para desabilitar todos os outros pipes.

In [14]:
# add the new label to ner
ner.add_label(LABEL)

# resume training
optimizer = nlp.resume_training()
move_names = list(ner.move_names)

# list of pipes you want to train
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

# list of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

Algumas considerações:

> a) Você tem que passar os exemplos pelo modelo pelo um número suficiente de iterações. Aqui, nós definimos 30 iterações.

> b) Lembre-se de ajustar o modelo de iterações de acordo com o desempenho. Além disso, antes de cada iteração, é melhor embaralhar os exemplos aleatoriamente por meio da função `random.shuffle()`. Isso garantirá que o modelo não faça generalizações com base na ordem dos exemplos.

> c) Os dados de treinamento devem ser passados em lotes. Você pode chamar a função `minibatch()` do `spaCy` sobre os exemplos de treinamento que retornarão seus dados em lotes. Um parâmetro da função de `minibatch` é `size`, denotando o tamanho do lote.

In [15]:
# importing requirements
from spacy.util import minibatch, compounding
import random

# Begin training by disabling other pipeline components
with nlp.disable_pipes(*other_pipes) :

  sizes = compounding(1.0, 4.0, 1.001)
  # training for 30 iterations     
  for itn in range(30):
    # shuffle examples before training
    random.shuffle(TRAIN_DATA)
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=sizes)
    # dictionary to store losses
    losses = {}
    for batch in batches:
      texts, annotations = zip(*batch)
      # Calling update() over the iteration
      nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
      print("Losses", losses)

Losses {'ner': 2.3995642220727347}
Losses {'ner': 10.304034546589032}
Losses {'ner': 13.305751696353019}
Losses {'ner': 15.496271063057412}
Losses {'ner': 19.265382820413173}
Losses {'ner': 25.35750985123493}
Losses {'ner': 32.311582422113226}
Losses {'ner': 37.68660711853605}
Losses {'ner': 45.81508636086088}
Losses {'ner': 51.72462280334745}
Losses {'ner': 59.403900008438235}
Losses {'ner': 63.18525014978595}
Losses {'ner': 67.31209391328903}
Losses {'ner': 4.856054702590484}
Losses {'ner': 9.289830486527933}
Losses {'ner': 15.575885139613035}
Losses {'ner': 18.49907881371533}
Losses {'ner': 24.884713728208823}
Losses {'ner': 31.58632288419963}
Losses {'ner': 35.11501042136834}
Losses {'ner': 40.84024588355706}
Losses {'ner': 46.706811799931444}
Losses {'ner': 57.1679939173936}
Losses {'ner': 62.54544261876609}
Losses {'ner': 64.48343722222297}
Losses {'ner': 71.91696004269569}
Losses {'ner': 2.9678623771214916}
Losses {'ner': 3.8934194141911576}
Losses {'ner': 5.1079882336607625}
Lo

Para cada iteração, o modelo é atualizado por meio do comando `nlp.update()`. Os parâmetros de `nlp.update()` são:

* `docs`: espera um lote de textos como entrada. Você pode passar cada lote para o método `zip`, que retornará lotes de texto e anotações.
     `
* `sgd`: você deve passar o otimizador que foi retornado por `resume_training()` aqui.

* `golds`: você pode passar as anotações que obtivemos por meio do método `zip` aqui.

* `drop`: representa a taxa de abandono.

* `losses`: um dicionário para conter as perdas em cada componente do pipeline. Crie um dicionário vazio e passe-o aqui.

A cada palavra, `update()` faz uma previsão. Em seguida, ele consulta as anotações para verificar se a previsão está correta. Se não estiver, ele ajusta os pesos para que a ação correta tenha uma pontuação mais alta na próxima vez.

O treinamento de nosso NER está concluído agora. Vamos testar se o NER pode identificar nossa nova entidade. Se não corresponder às suas expectativas, tente incluir mais exemplos de treinamento.

In [16]:
doc = nlp("I ate Sushi yesterday. Maggi is a common fast food")

displacy.render(doc, style='ent', jupyter=True)

Observe a saída acima. O modelo identificou corretamente os itens `FOOD`. Além disso, observe que não passamos "Maggi" como um exemplo de treinamento para a modelo. Ainda assim, com base na similaridade de contexto, o modelo identificou "Maggi" também como `FOOD`. 

>
**Este é um requisito importante! Nosso modelo não deve apenas memorizar os exemplos de treinamento. Deve aprender com eles e generalizar para novos exemplos.**

Depois de encontrar o desempenho do modelo satisfatório, você pode salvar o modelo atualizado no diretório usando o comando `to_disk`. Você também pode carregar o modelo do diretório a qualquer momento, passando o caminho do diretório para a função `spacy.load()`.

In [17]:
# output directory
from pathlib import Path

output_dir=Path('/content/modelo')

In [18]:
# saving the model to the output directory
if not output_dir.exists():
  output_dir.mkdir()
nlp.meta['name'] = 'my_ner'  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to /content/modelo


In [19]:
# loading the model from the directory
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)

#assert nlp2.get_pipe("ner").move_names == move_names
doc2 = nlp2('Dosa is an extremely famous south Indian dish')

for ent in doc2.ents:
  print(ent.label_, ent.text)

Loading from /content/modelo
FOOD Dosa


In [20]:
displacy.render(doc2, style='ent', jupyter=True)

**Referência**:

*How to Train spaCy to Autodetect New Entities (NER) [Complete Guide]*
> https://www.machinelearningplus.com/nlp/training-custom-ner-model-in-spacy/